In [1]:
import keras
import scipy
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import layers

import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from gplearn.genetic import SymbolicRegressor

from numba import jit, float64, int64

plt.rc('font', size=12)          # controls default text sizes
plt.rc('axes', titlesize=12)     # fontsize of the axes title
plt.rc('axes', labelsize=15)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=12)    # fontsize of the tick labels
plt.rc('ytick', labelsize=12)    # fontsize of the tick labels
plt.rc('legend', fontsize=15)    # legend fontsize

tf.keras.backend.clear_session()

In [2]:
@jit(nopython=True)
def Race(v, b, ndt=0, dt=0.001):
    x = np.array([0.0, 0.0])
    rt = 0
    ch = -1
    stop = False
    while not stop:
        x[0] += v[0]*dt + np.random.normal(0, 1)*np.sqrt(dt)
        x[1] += v[1]*dt + np.random.normal(0, 1)*np.sqrt(dt)
        rt += dt
        if x[0]>= b and x[1]<b:
            ch = 0.0
            stop = True
        elif x[1]>= b and x[0]<b:
            ch = 1.0
            stop = True
        elif x[0]>= b and x[1]>=b:
            rt = 0
            x = np.array([0.0, 0.0])
    return rt+ndt, ch

In [3]:
data_df = pd.read_csv('../../_00_data/Kr2010_prep.csv', index_col=0)

min_rt = data_df.groupby(by='sbj')['rt'].min()

tr_df, ts_df = data_df.copy(), data_df.copy()

obs_train = np.asarray(data_df[['rt', 'Right', 'sbj']])
X_train = np.asarray(data_df[['item_value_1', 'item_value_2', 'gaze_1', 'gaze_2']])

obs_test = np.asarray(data_df[['rt', 'Right', 'sbj']])
X_test = np.asarray(data_df[['item_value_1', 'item_value_2', 'gaze_1', 'gaze_2']])

print(X_train.shape, X_test.shape)

(3522, 4) (3522, 4)


In [4]:
def fpt(t, b, v):
    return tf.cast(tf.math.greater(t, 0), tf.float32)*(b/(2*np.pi*t**3)**(1/2) * keras.backend.exp(-1/(2*t) * (v*t-b)**2)) + tf.cast(tf.math.less_equal(t, 0), tf.float32)*0.1**10

def phi(x):
    return 0.5*tf.math.erf(x/tf.sqrt(2.0)) + 0.5

def cdf(t, b, v):
    return tf.cast(tf.math.greater(t, 0), tf.float32)*(phi((v*t-b)/tf.sqrt(t)) + tf.exp(2*v*b)*phi((-v*t-b)/tf.sqrt(t))) + (tf.cast(tf.math.less_equal(t, 0), tf.float32)*0)

def G2(data, pred):

    sbj_indices = tf.cast(data[:, 2], tf.int32)

    # Gather values directly using TensorFlow operations
    gathered_min_rt = tf.gather(min_rt, sbj_indices)
    sigmoid_weights = tf.gather(model.layers[-1].weights[-1], sbj_indices)
    ndt = gathered_min_rt * tf.math.sigmoid(sigmoid_weights)

    thr_weights = tf.gather(model.layers[-1].weights[-2], sbj_indices)
    thr = tf.math.log(1 + tf.exp(thr_weights))

    scl_weights = tf.gather(model.layers[-1].weights[-3], sbj_indices)
    scl = tf.math.log(1 + tf.exp(scl_weights))

    ll1 = (1 - data[:, 1]) * fpt(data[:, 0]-ndt, thr, scl*pred[:, 0]) * (1.0 - cdf(data[:, 0]-ndt, thr, scl*pred[:, 1]))
    ll2 = (data[:, 1]) * fpt(data[:, 0]-ndt, thr, scl*pred[:, 1]) * (1.0 - cdf(data[:, 0]-ndt, thr, scl*pred[:, 0]))

    return -2*tf.reduce_sum(tf.math.log(tf.maximum(ll1 + ll2, 1e-14)), axis=-1)

In [5]:
class Out_layer(keras.layers.Layer):
    def __init__(self, units=2, input_dim=32, n_sbj_prms=10):
        super().__init__()
        self.w = self.add_weight(shape=(input_dim, units), initializer="random_normal", trainable=True)
        self.b = self.add_weight(shape=(units,), initializer="uniform", trainable=True)
        
        self.scaling = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)
        self.threshold = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)
        self.non_decision_time = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)

    def call(self, inputs):
        return tf.math.log(1 + tf.exp(tf.matmul(inputs, self.w) + self.b))
    
def make_model(layers_neuron, input_shape, output_shape, activation, n_sbj_prms):
    keras.backend.clear_session()

    model = keras.Sequential()
    model.add(layers.Input(shape=(input_shape,)))
    
    for l in layers_neuron:
        model.add(layers.Dense(l, activation=activation))
    
    model.add(Out_layer(units=output_shape, input_dim=layers_neuron[-1], n_sbj_prms=n_sbj_prms))

    print(model.summary())
    
    return model

In [6]:
layers_neuron = [10, 10, 8, 5]
input_shape = 4
output_shape = 2
activation = 'tanh'
n_sbj = data_df.sbj.nunique()

model = make_model(layers_neuron, input_shape, output_shape, activation, n_sbj)

model.build()
model.compile(optimizer="RMSprop", loss=G2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                50        
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 8)                 88        
                                                                 
 dense_3 (Dense)             (None, 5)                 45        
                                                                 
 out_layer (Out_layer)       (None, 2)                 129       
                                                                 
Total params: 422 (1.65 KB)
Trainable params: 422 (1.65 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [7]:
sbj_indices = tf.cast(obs_train[:, 2], tf.int32)
obs_train = tf.cast(obs_train, tf.float32)
X_train = tf.convert_to_tensor(X_train)
X_train = tf.cast(X_train, tf.float32)

min_rt = tf.convert_to_tensor(min_rt)
min_rt = tf.cast(min_rt, tf.float32)

In [8]:
history = model.fit(X_train, 
                    obs_train, 
                    shuffle=False, 
                    batch_size=X_train.shape[0], 
                    epochs=4000)

Epoch 1/4000
1/1 [==============================] - 1s 565ms/step - loss: 21553.8672
Epoch 2/4000
1/1 [==============================] - 0s 2ms/step - loss: 21457.5078
Epoch 3/4000
1/1 [==============================] - 0s 2ms/step - loss: 21387.8184
Epoch 4/4000
1/1 [==============================] - 0s 3ms/step - loss: 21328.4141
Epoch 5/4000
1/1 [==============================] - 0s 2ms/step - loss: 21274.7363
Epoch 6/4000
1/1 [==============================] - 0s 3ms/step - loss: 21224.8223
Epoch 7/4000
1/1 [==============================] - 0s 3ms/step - loss: 21177.7109
Epoch 8/4000
1/1 [==============================] - 0s 4ms/step - loss: 21132.8633
Epoch 9/4000
1/1 [==============================] - 0s 3ms/step - loss: 21089.9062
Epoch 10/4000
1/1 [==============================] - 0s 3ms/step - loss: 21048.5703
Epoch 11/4000
1/1 [==============================] - 0s 3ms/step - loss: 21008.6387
Epoch 12/4000
1/1 [==============================] - 0s 3ms/step - loss: 20969.9180

1/1 [==============================] - 0s 2ms/step - loss: 18740.9375
Epoch 99/4000
1/1 [==============================] - 0s 3ms/step - loss: 18722.3906
Epoch 100/4000
1/1 [==============================] - 0s 3ms/step - loss: 18703.9629
Epoch 101/4000
1/1 [==============================] - 0s 3ms/step - loss: 18685.6602
Epoch 102/4000
1/1 [==============================] - 0s 3ms/step - loss: 18667.4746
Epoch 103/4000
1/1 [==============================] - 0s 2ms/step - loss: 18649.4102
Epoch 104/4000
1/1 [==============================] - 0s 2ms/step - loss: 18631.4609
Epoch 105/4000
1/1 [==============================] - 0s 3ms/step - loss: 18613.6289
Epoch 106/4000
1/1 [==============================] - 0s 3ms/step - loss: 18595.9102
Epoch 107/4000
1/1 [==============================] - 0s 3ms/step - loss: 18578.3105
Epoch 108/4000
1/1 [==============================] - 0s 3ms/step - loss: 18560.8164
Epoch 109/4000
1/1 [==============================] - 0s 3ms/step - loss: 18543.4

1/1 [==============================] - 0s 2ms/step - loss: 17367.7207
Epoch 195/4000
1/1 [==============================] - 0s 3ms/step - loss: 17356.5254
Epoch 196/4000
1/1 [==============================] - 0s 3ms/step - loss: 17345.3730
Epoch 197/4000
1/1 [==============================] - 0s 4ms/step - loss: 17334.2637
Epoch 198/4000
1/1 [==============================] - 0s 3ms/step - loss: 17323.2051
Epoch 199/4000
1/1 [==============================] - 0s 2ms/step - loss: 17312.1875
Epoch 200/4000
1/1 [==============================] - 0s 3ms/step - loss: 17301.2129
Epoch 201/4000
1/1 [==============================] - 0s 2ms/step - loss: 17290.2852
Epoch 202/4000
1/1 [==============================] - 0s 3ms/step - loss: 17279.4004
Epoch 203/4000
1/1 [==============================] - 0s 3ms/step - loss: 17268.5547
Epoch 204/4000
1/1 [==============================] - 0s 2ms/step - loss: 17257.7559
Epoch 205/4000
1/1 [==============================] - 0s 3ms/step - loss: 17246.

1/1 [==============================] - 0s 2ms/step - loss: 16456.2559
Epoch 291/4000
1/1 [==============================] - 0s 3ms/step - loss: 16448.1211
Epoch 292/4000
1/1 [==============================] - 0s 2ms/step - loss: 16440.0059
Epoch 293/4000
1/1 [==============================] - 0s 2ms/step - loss: 16431.9121
Epoch 294/4000
1/1 [==============================] - 0s 3ms/step - loss: 16423.8398
Epoch 295/4000
1/1 [==============================] - 0s 2ms/step - loss: 16415.7891
Epoch 296/4000
1/1 [==============================] - 0s 3ms/step - loss: 16407.7578
Epoch 297/4000
1/1 [==============================] - 0s 2ms/step - loss: 16399.7461
Epoch 298/4000
1/1 [==============================] - 0s 2ms/step - loss: 16391.7598
Epoch 299/4000
1/1 [==============================] - 0s 3ms/step - loss: 16383.7891
Epoch 300/4000
1/1 [==============================] - 0s 2ms/step - loss: 16375.8418
Epoch 301/4000
1/1 [==============================] - 0s 2ms/step - loss: 16367.

1/1 [==============================] - 0s 2ms/step - loss: 15740.0264
Epoch 387/4000
1/1 [==============================] - 0s 2ms/step - loss: 15732.9766
Epoch 388/4000
1/1 [==============================] - 0s 3ms/step - loss: 15725.9258
Epoch 389/4000
1/1 [==============================] - 0s 2ms/step - loss: 15718.8740
Epoch 390/4000
1/1 [==============================] - 0s 2ms/step - loss: 15711.8232
Epoch 391/4000
1/1 [==============================] - 0s 2ms/step - loss: 15704.7715
Epoch 392/4000
1/1 [==============================] - 0s 2ms/step - loss: 15697.7168
Epoch 393/4000
1/1 [==============================] - 0s 3ms/step - loss: 15690.6641
Epoch 394/4000
1/1 [==============================] - 0s 2ms/step - loss: 15683.6074
Epoch 395/4000
1/1 [==============================] - 0s 2ms/step - loss: 15676.5479
Epoch 396/4000
1/1 [==============================] - 0s 3ms/step - loss: 15669.4873
Epoch 397/4000
1/1 [==============================] - 0s 2ms/step - loss: 15662.

...

1/1 [==============================] - 0s 3ms/step - loss: 9416.0078
Epoch 3551/4000
1/1 [==============================] - 0s 3ms/step - loss: 9413.5459
Epoch 3552/4000
1/1 [==============================] - 0s 4ms/step - loss: 9413.5732
Epoch 3553/4000
1/1 [==============================] - 0s 4ms/step - loss: 9412.0693
Epoch 3554/4000
1/1 [==============================] - 0s 3ms/step - loss: 9413.3701
Epoch 3555/4000
1/1 [==============================] - 0s 4ms/step - loss: 9410.3154
Epoch 3556/4000
1/1 [==============================] - 0s 3ms/step - loss: 9411.1768
Epoch 3557/4000
1/1 [==============================] - 0s 4ms/step - loss: 9413.5557
Epoch 3558/4000
1/1 [==============================] - 0s 3ms/step - loss: 9412.5264
Epoch 3559/4000
1/1 [==============================] - 0s 3ms/step - loss: 9411.4883
Epoch 3560/4000
1/1 [==============================] - 0s 3ms/step - loss: 9411.5771
Epoch 3561/4000
1/1 [==============================] - 0s 4ms/step - loss: 9412.3

1/1 [==============================] - 0s 3ms/step - loss: 9406.4521
Epoch 3647/4000
1/1 [==============================] - 0s 3ms/step - loss: 9405.4092
Epoch 3648/4000
1/1 [==============================] - 0s 3ms/step - loss: 9403.8164
Epoch 3649/4000
1/1 [==============================] - 0s 3ms/step - loss: 9407.2891
Epoch 3650/4000
1/1 [==============================] - 0s 6ms/step - loss: 9406.6670
Epoch 3651/4000
1/1 [==============================] - 0s 3ms/step - loss: 9408.8115
Epoch 3652/4000
1/1 [==============================] - 0s 4ms/step - loss: 9408.4072
Epoch 3653/4000
1/1 [==============================] - 0s 4ms/step - loss: 9421.6895
Epoch 3654/4000
1/1 [==============================] - 0s 4ms/step - loss: 9431.1279
Epoch 3655/4000
1/1 [==============================] - 0s 3ms/step - loss: 9483.1309
Epoch 3656/4000
1/1 [==============================] - 0s 3ms/step - loss: 9412.9678
Epoch 3657/4000
1/1 [==============================] - 0s 3ms/step - loss: 9408.3

1/1 [==============================] - 0s 3ms/step - loss: 9401.5166
Epoch 3743/4000
1/1 [==============================] - 0s 3ms/step - loss: 9399.1084
Epoch 3744/4000
1/1 [==============================] - 0s 2ms/step - loss: 9404.4404
Epoch 3745/4000
1/1 [==============================] - 0s 3ms/step - loss: 9403.0391
Epoch 3746/4000
1/1 [==============================] - 0s 3ms/step - loss: 9415.7490
Epoch 3747/4000
1/1 [==============================] - 0s 2ms/step - loss: 9412.4629
Epoch 3748/4000
1/1 [==============================] - 0s 3ms/step - loss: 9409.9902
Epoch 3749/4000
1/1 [==============================] - 0s 3ms/step - loss: 9404.8174
Epoch 3750/4000
1/1 [==============================] - 0s 2ms/step - loss: 9405.4326
Epoch 3751/4000
1/1 [==============================] - 0s 2ms/step - loss: 9403.9619
Epoch 3752/4000
1/1 [==============================] - 0s 2ms/step - loss: 9407.4238
Epoch 3753/4000
1/1 [==============================] - 0s 3ms/step - loss: 9408.0

1/1 [==============================] - 0s 2ms/step - loss: 9397.6035
Epoch 3839/4000
1/1 [==============================] - 0s 5ms/step - loss: 9396.8486
Epoch 3840/4000
1/1 [==============================] - 0s 5ms/step - loss: 9397.6211
Epoch 3841/4000
1/1 [==============================] - 0s 5ms/step - loss: 9395.6377
Epoch 3842/4000
1/1 [==============================] - 0s 3ms/step - loss: 9399.7861
Epoch 3843/4000
1/1 [==============================] - 0s 3ms/step - loss: 9398.6123
Epoch 3844/4000
1/1 [==============================] - 0s 2ms/step - loss: 9404.0205
Epoch 3845/4000
1/1 [==============================] - 0s 3ms/step - loss: 9403.9697
Epoch 3846/4000
1/1 [==============================] - 0s 3ms/step - loss: 9407.2764
Epoch 3847/4000
1/1 [==============================] - 0s 3ms/step - loss: 9404.6797
Epoch 3848/4000
1/1 [==============================] - 0s 3ms/step - loss: 9405.4912
Epoch 3849/4000
1/1 [==============================] - 0s 2ms/step - loss: 9400.4

1/1 [==============================] - 0s 2ms/step - loss: 9394.7422
Epoch 3935/4000
1/1 [==============================] - 0s 2ms/step - loss: 9393.8096
Epoch 3936/4000
1/1 [==============================] - 0s 2ms/step - loss: 9401.0986
Epoch 3937/4000
1/1 [==============================] - 0s 2ms/step - loss: 9402.6182
Epoch 3938/4000
1/1 [==============================] - 0s 2ms/step - loss: 9402.6777
Epoch 3939/4000
1/1 [==============================] - 0s 2ms/step - loss: 9398.0371
Epoch 3940/4000
1/1 [==============================] - 0s 2ms/step - loss: 9402.1553
Epoch 3941/4000
1/1 [==============================] - 0s 2ms/step - loss: 9399.8242
Epoch 3942/4000
1/1 [==============================] - 0s 2ms/step - loss: 9399.2061
Epoch 3943/4000
1/1 [==============================] - 0s 2ms/step - loss: 9396.7510
Epoch 3944/4000
1/1 [==============================] - 0s 2ms/step - loss: 9396.0635
Epoch 3945/4000
1/1 [==============================] - 0s 2ms/step - loss: 9398.1

In [9]:
pred_v = model.predict(X_test)
threshold = np.asanyarray(tf.math.log(1 + tf.exp(model.layers[-1].weights[-2])))
ndt = np.asanyarray(np.asanyarray(min_rt)*tf.math.sigmoid(model.layers[-1].weights[-1]))

np.save('input.npy', X_test)
np.save('output.npy', pred_v)

111/111 [==============================] - 0s 274us/step


In [10]:
pred_bhv = {'sbj': [],
            'rt': [],
            'ch': [],
            'value_diff': [],
            'gaze_diff': []}

scaling = np.asanyarray(tf.math.log(1 + tf.exp(model.layers[-1].weights[-3])))
scaling = np.asanyarray([scaling[i] for i in ts_df['sbj']])

for n in range(20):
    for s in range(ts_df.shape[0]):
        rt, ch = Race(scaling[s]*pred_v[s,:], threshold[ts_df.sbj[s]], ndt[ts_df.sbj[s]])
        pred_bhv['sbj'].append(ts_df.sbj[s])
        pred_bhv['rt'].append(rt)
        pred_bhv['ch'].append(ch)
        pred_bhv['value_diff'].append(ts_df.item_value_2[s] - ts_df.item_value_1[s])
        pred_bhv['gaze_diff'].append(ts_df['Dwell Time Advantage (R - L)'][s])
        
pred_bhv = pd.DataFrame(pred_bhv)
pred_bhv.to_csv('Kr2010_complete.csv')

In [11]:
# est_gp = SymbolicRegressor(population_size=5000,
#                            function_set=['add', 'sub', 'mul', 'div'],
#                            generations=50, stopping_criteria=0.01,
#                            p_crossover=0.7, p_subtree_mutation=0.1,
#                            p_hoist_mutation=0.05, p_point_mutation=0.1,
#                            max_samples=0.9, verbose=1,
#                            parsimony_coefficient=0.01, random_state=0)
# est_gp.fit(X_train, pred_v[:, 0]-pred_v[:, 1])
# print(est_gp._program)